In [1]:
#!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import uuid   # Unique identifier
import os
import time
import psycopg2
import pandas as pd
hostname = 'localhost'
database = 'employeeDB'
username = 'postgres'
pwd = 'admin'
port_id = 5432
conn = psycopg2.connect(host=hostname, dbname=database, user=username, password=pwd, port=port_id)
cur = conn.cursor()

C:\Users\Digvijay Singh Dhank\anaconda3\envs\drowsy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\Digvijay Singh Dhank/.cache\torch\hub\master.zip
YOLOv5  2022-5-30 Python-3.8.13 torch-1.8.2+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [25]:
cap = cv2.VideoCapture(0)
var = ""
safety_score = 99;
location_name = "street2"
location_id = 123
while cap.isOpened():
    ret, frame = cap.read()
    cv2.rectangle(frame, (0, 0), (525, 73), (245, 117, 16), -1)
    cv2.putText(frame, "Location Name:", (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Location I.D.:",
                        (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, location_name, (180, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, str(location_id),
                        (180, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    df = results.pandas().xyxy[0]
    d_count=df['name'].str.contains('threat').sum()
    if d_count>0:
        safety_score = safety_score - d_count
        score = safety_score.item()
        sql = "UPDATE employee SET score = array_append(score, %s) WHERE location = %s"
        var = (score,location_name,)
        cur.execute(sql, var)
    if d_count==1:
             print("threat")
    if cv2.waitKey(10) & 0xFF == ord('q'):
        conn.commit()
        break
cap.release()
cv2.destroyAllWindows()

threat
threat
threat
threat
threat
threat
threat
threat
threat
threat
threat
threat
threat


In [22]:
# cur.close()
# conn.close()

In [28]:
my_table    = pd.read_sql('select * from employee', conn)
print(my_table)

   id    location                                              score
0   1         ATM  [100, 99, 94, 93, 92, 91, 90, 89, 88, 87, 86, ...
1   2   Warehouse  [100, 99, 98, 69, 65, 50, 45, 45, 45, 45, 45, ...
2   3      Street  [200, 97, 5, 10, 94, 93, 92, 91, 90, 89, 88, 8...
3   4     street2  [100, 98, 97, 96, 93, 90, 88, 86, 85, 84, 82, ...
4   5  warehouse2                                              [100]


C:\Users\Digvijay Singh Dhank\anaconda3\envs\drowsy\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
